In [13]:
import os

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

import random

# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")

#import sistemas
import sei
import functions
import page
from page import *
import helpers
import sistemas

% reload_ext autoreload
% autoreload 2

In [ ]:
COLS = ["CPF", "Nome", "Fistel", "Validade", "End_Sede", "Nome_RF", "End_Sede",
        "End_Correspondencia", "End_Completo", "CPF_RF", ]

In [39]:
processo = "53504.005936/2018-50"

In [4]:
gc = functions.authenticate_google("files/anatel.json")
wb = gc.open(title="Expedidos")
sht = wb.get_worksheet(0)
df = gs.get_as_dataframe(sht, dtype=str)

In [9]:
def extrai_pares_tabulação(source):
    trs = source.find_all('tr')
    dados = {}
    i = 1
    for tr in trs:        
        td = tr.find_all('td', string=True)
        label = tr.find_all('label', string=True)
        
        i = 1
        for field, result in zip(td, label):
            field, result = field.text[:-1], result.text
            if field in dados:
                field = field + "_" + str(i + 1)
            dados[field] = result 
       
    return dados

In [15]:
usr = "rsilva"
pwd = "Savorthemom3nts"

driver = webdriver.Ie()
scpx = sistemas.Scpx(driver)

In [ ]:
vals = []

for i, values in df[df["Processo"] == processo].iterrows():
    fistel = values.FISTEL
    while len(fistel) < 11:
        fistel = '0' + fistel
    scpx.consulta(fistel,tipo_id='id_fistel')
    source = soup(scpx.driver.page_source, "lxml")
    dados = extrai_pares_tabulação(source)
    
    scpx.wait_for_element_to_click((By.ID, "botaoFlatPróximo")).click()
    val_rf = source.find(id='labelDataValidadeRadioFrequencia')
    
    #source = soup(scpx.driver.page_source, "lxml")
    #dados = {**dados, **extrai_pares_tabulação(source)}
                                               
    #proximo = scpx.wait_for_element_to_click((By.ID, "botaoFlatEstação"))
    #proximo.click()
    
    #source = soup(scpx.driver.page_source, "lxml")
    #dados = {**dados, **extrai_pares_tabulação(source)}
    
    print("Nome: {}".format(values['NOME']))
    print("Validade de Radiofrequência: {}".format(val_rf.text))
    print("Data de Validade Licença: {}\n".format(dados['Data Validade Licença']))
    
    vals.append(val_rf) 
    
df.loc[df["Processo"] == processo, "Validade_RF"] = vals

Nome: ALEXANDRE CARLOS CALADO
Validade de Radiofrequência: 06/09/2017
Data de Validade Licença: 06/09/2017

Nome: CELSO DONIZETE SALOMAO
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: EDGAR HENRIQUE NERY
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: EDUARDO CAMPOS PARDO
Validade de Radiofrequência: 06/06/2017
Data de Validade Licença: 31/03/2008

Nome: FABIANO SABINO DA SILVA
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: HELIO DE OLIVEIRA NUNES
Validade de Radiofrequência: 13/06/2037
Data de Validade Licença: 13/06/2037

Nome: HENRIQUE CREPALDI INFORZATO
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: JUAREZ LOYOLA JUNIOR
Validade de Radiofrequência: 30/05/2017
Data de Validade Licença: 30/05/2017

Nome: MARCELO HUERTAS RODRIGUES
Validade de Radiofrequência: 30/05/2017
Data de Validade Licença: 30/05/2017

Nome: MARCELO JOSE CARNEIRO
Valid